# Miniflow 
A small DNN implementation with differentiable graphs

In [1]:
import numpy as np

## Node class

In [2]:
class Node(object):
    def __init__(self, inbound_nodes=[]):
        # inbound and outbound nodes
        self.inbound_nodes = inbound_nodes
        self.outbound_nodes = []
        
        # for each inbound node, add this as it's outbound node
        for n in self.inbound_nodes:
            n.outbound_nodes.append(self)
        
        # value of the node
        self.value = None 
        
        # keys are inputs to this node and values are partials wrt to inputs of node
        self.gradients = {}
        
    def forward(self):
        """
        Forward propogation
        
        Compute the value based on input nodes and store it in self.value
        """
        
        raise NotImplemented
    def backward(self):
        """
        Every node that uses this class as a base class will
        need to define its own `backward` method.
        """
        raise NotImplementedError
    

In [3]:
class Input(Node):
    def __init__(self):
        # input node has no inbound nodes
        Node.__init__(self)
        
    def forward(self, value=None):
        """
        input node is the only node when value can be passed as an argument for forward()
        """
        if value is not None:
            self.value = value
            
    def backward(self):
        # input has no inputs so the derivative is zero
        self.gradients = {self: 0}
        # weights and biases may be inputs so need to sum the gradient from output gradients
        for n in self.outbound_nodes:
            grad_cost = n.gradients[self]
            self.gradients[self] += grad_cost * 1

In [4]:
class Add(Node):
    def __init__(self, *inputs):
        Node.__init__(self, inputs)
        
    def forward(self):
        self.value = 0
        for n in self.inbound_nodes:
            self.value += n.value

In [5]:
class Mul(Node):
    def __init__(self, *inputs):
        Node.__init__(self, inputs)
        
    def forward(self):
        self.value = 1
        for n in self.inbound_nodes:
            self.value *= n.value

## Topological sort

Sorting nodes and flattening the graph such that input dependencies for each node is resolved before trying to run it's calculations. Here using Kahn's Algorithm 

In [6]:
def topological_sort(feed_dict):
    """
    sorts generic nodes in topological order
    'feed_dict`: a dictionary where the key is an `Input` node and value is the value of node
    return list of sorted nodes
    """

    # input nodes
    input_nodes = [n for n in feed_dict.keys()]
    
    # sets up the graph by creating connections for each node
    G = {}
    nodes = [n for n in input_nodes]
    
    # create a graph in G
    # while nodes is non-empty
    while len(nodes) > 0:
        # remove a node n from nodes
        n = nodes.pop(0)
        if n not in G:
            # add n to G
            G[n] = {'in': set(), 'out':set()}
        # for each output on n
        for m in n.outbound_nodes:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)

    # list of sorted elements
    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()
        
        # if input node, set value
        if isinstance(n, Input):
            n.value = feed_dict[n]
            
        L.append(n)
        # remove edges
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            # if no other incoming edges add to S
            if len(G[m]['in']) == 0:
                S.add(m)
                
    return L

## forward pass
runs the network and returns the output

In [7]:
def forward_pass(output_node, sorted_nodes):
    """
    performs forward pass through list of sorted nodes
    
    return output node's value
    """
    
    for n in sorted_nodes:
        n.forward()
    
    return output_node.value

In [8]:
x, y, z = Input(), Input(), Input()

f = Add(x,y)
g = Mul(z,f)

feed_dict = {x: 10, y:5, z:10}

sorted_nodes = topological_sort(feed_dict)
output = forward_pass(g, sorted_nodes)

print(output)

150


## forward and backwards pass

In [9]:
def forward_and_backward(graph):
    """
    Performs a forward pass and a backward pass through a list of sorted nodes.

    Arguments:

        `graph`: The result of calling `topological_sort`.
    """
    # Forward pass
    for n in graph:
        n.forward()

    # Backward pass
    for n in graph[::-1]:
        n.backward()

## Linear node
Takes inputs, weights and biases and returns output

In [10]:
class Linear(Node):
    def __init__(self, inputs, weights, bias):
        Node.__init__(self, [inputs, weights, bias])

        # NOTE: The weights and bias properties here are not
        # numbers, but rather references to other nodes.
        # The weight and bias values are stored within the
        # respective nodes.

    def forward(self):
        inputs = self.inbound_nodes[0].value
        weights = self.inbound_nodes[1].value
        bias = self.inbound_nodes[2].value
        
        assert(len(inputs) == len(weights))
        
        self.value = 0
        for i in range(len(inputs)):
            self.value += inputs[i] * weights[i]
        self.value += bias
    

In [11]:
inputs, weights, bias = Input(), Input(), Input()

f = Linear(inputs, weights, bias)

feed_dict = {
    inputs: [6,14,3],
    weights: [0.5,0.25,1.4],
    bias: 2
}

graph = topological_sort(feed_dict)
output = forward_pass(f, graph)

print(output)

12.7


## Linear node with matrices

In [12]:
class Linear(Node):
    def __init__(self, inputs, weights, bias):
        Node.__init__(self, [inputs, weights, bias])

        # NOTE: The weights and bias properties here are not
        # numbers, but rather references to other nodes.
        # The weight and bias values are stored within the
        # respective nodes.

    def forward(self):
        X = np.array(self.inbound_nodes[0].value)
        W = np.array(self.inbound_nodes[1].value)
        b = np.array(self.inbound_nodes[2].value)
                
        self.value = X.dot( W ) + b
    
    def backward(self):
        # initialize a partial for each inbound nodes
        self.gradients = {n: np.zeros_like(n.value) for n in self.inbound_nodes}
        for n in self.outbound_nodes:
            grad_cost = n.gradients[self]
            # partial wrt node's inputs 
            self.gradients[self.inbound_nodes[0]] += np.dot(grad_cost, self.inbound_nodes[1].value.T)
            # partial wrt node's weight
            self.gradients[self.inbound_nodes[1]] += np.dot(self.inbound_nodes[0].value.T, grad_cost)
            # partial wrt node's bias
            self.gradients[self.inbound_nodes[2]] += np.sum(grad_cost, axis=0, keepdims=False)

In [13]:
X, W, b = Input(), Input(), Input()

f = Linear(X, W, b)

X_ = np.array([[-1., -2.], [-1, -2]])
W_ = np.array([[2., -3], [2., -3]])
b_ = np.array([-3., -5])

feed_dict = {X: X_, W: W_, b: b_}

graph = topological_sort(feed_dict)
output = forward_pass(f, graph)

"""
Output should be:
[[-9., 4.],
[-9., 4.]]
"""
print(output)

[[-9.  4.]
 [-9.  4.]]


## Sigmoid

In [14]:
class Sigmoid(Node):
    """
    You need to fix the `_sigmoid` and `forward` methods.
    """
    def __init__(self, node):
        Node.__init__(self, [node])

    def _sigmoid(self, x):
        """
        This method is separate from `forward` because it
        will be used later with `backward` as well.

        `x`: A numpy array-like object.

        Return the result of the sigmoid function.

        Your code here!
        """
        return 1 / (1 + np.exp(-x))


    def forward(self):
        """
        Set the value of this node to the result of the
        sigmoid function, `_sigmoid`.

        Your code here!
        """
        # This is a dummy value to prevent numpy errors
        # if you test without changing this method.
        self.value = self._sigmoid(self.inbound_nodes[0].value)
        
    def backward(self):
        self.gradients = {n: np.zeros_like(n.value) for n in self.inbound_nodes}
        
        for n in self.outbound_nodes:
            grad_cost = n.gradients[self]
            sigmoid = self.value
            self.gradients[self.inbound_nodes[0]] += sigmoid * (1-sigmoid) * grad_cost

In [15]:
X, W, b = Input(), Input(), Input()

f = Linear(X, W, b)
g = Sigmoid(f)

X_ = np.array([[-1., -2.], [-1, -2]])
W_ = np.array([[2., -3], [2., -3]])
b_ = np.array([-3., -5])

feed_dict = {X: X_, W: W_, b: b_}

graph = topological_sort(feed_dict)
output = forward_pass(g, graph)

"""
Output should be:
[[  1.23394576e-04   9.82013790e-01]
 [  1.23394576e-04   9.82013790e-01]]
"""
print(output)

[[  1.23394576e-04   9.82013790e-01]
 [  1.23394576e-04   9.82013790e-01]]


## Cost function

In [16]:
class MSE(Node):
    def __init__(self, y, a):
        """
        The mean squared error cost function.
        Should be used as the last node for a network.
        """
        # Call the base class' constructor.
        Node.__init__(self, [y, a])

    def forward(self):
        """
        Calculates the mean squared error.
        """
        # NOTE: We reshape these to avoid possible matrix/vector broadcast
        # errors.
        #
        # For example, if we subtract an array of shape (3,) from an array of shape
        # (3,1) we get an array of shape(3,3) as the result when we want
        # an array of shape (3,1) instead.
        #
        # Making both arrays (3,1) insures the result is (3,1) and does
        # an elementwise subtraction as expected.
        y = self.inbound_nodes[0].value.reshape(-1, 1)
        a = self.inbound_nodes[1].value.reshape(-1, 1)
        self.m = self.inbound_nodes[0].value.shape[0]
        self.diff = y - a
        self.value = np.mean(self.diff**2)
    
    def backward(self):
        self.gradients[self.inbound_nodes[0]] = (2/self.m) * self.diff
        self.gradients[self.inbound_nodes[1]] = (-2/self.m) * self.diff

In [17]:
def forward_pass(graph):
    """
    Performs a forward pass through a list of sorted Nodes.

    Arguments:

        `graph`: The result of calling `topological_sort`.
    """
    # Forward pass
    for n in graph:
        n.forward()


In [18]:
y, a = Input(), Input()
cost = MSE(y, a)

y_ = np.array([1, 2, 3])
a_ = np.array([4.5, 5, 10])

feed_dict = {y: y_, a: a_}
graph = topological_sort(feed_dict)

# forward pass
forward_pass(graph)

"""
Expected output

23.4166666667
"""
print(cost.value)

23.4166666667


## Backprop

In [19]:
X, W, b = Input(), Input(), Input()
y = Input()
f = Linear(X, W, b)
a = Sigmoid(f)
cost = MSE(y, a)

X_ = np.array([[-1., -2.], [-1, -2]])
W_ = np.array([[2.], [3.]])
b_ = np.array([-3.])
y_ = np.array([1, 2])

feed_dict = {
    X: X_,
    y: y_,
    W: W_,
    b: b_,
}

graph = topological_sort(feed_dict)
forward_and_backward(graph)
# return the gradients for each Input
gradients = [t.gradients[t] for t in [X, y, W, b]]

"""
Expected output

[array([[ -3.34017280e-05,  -5.01025919e-05],
       [ -6.68040138e-05,  -1.00206021e-04]]), array([[ 0.9999833],
       [ 1.9999833]]), array([[  5.01028709e-05],
       [  1.00205742e-04]]), array([ -5.01028709e-05])]
"""
print(gradients)

[array([[ -3.34017280e-05,  -5.01025919e-05],
       [ -6.68040138e-05,  -1.00206021e-04]]), array([[ 0.9999833],
       [ 1.9999833]]), array([[  5.01028709e-05],
       [  1.00205742e-04]]), array([ -5.01028709e-05])]


## SGD

In [20]:
def sgd_update(trainables, learning_rate=1e-2):
    for t in trainables:
        t.value -= t.gradients[t] * learning_rate

In [21]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.utils import shuffle, resample

In [22]:
# load data
data = load_boston()
X_ = data['data']
Y_ = data['target']

In [23]:
# normalize data
X_ = (X_ - np.mean(X_, axis=0)) / np.std(X_, axis=0)

In [24]:
# initialize
n_features = X_.shape[1]
n_hidden = 10
W1_ = np.random.randn(n_features, n_hidden)
b1_ = np.zeros(n_hidden)
W2_ = np.random.randn(n_hidden, 1)
b2_ = np.zeros(1)

In [25]:
# build network
X, y = Input(), Input()
W1, b1 = Input(), Input()
W2, b2 = Input(), Input()

l1 = Linear(X, W1, b1)
s1 = Sigmoid(l1)
l2 = Linear(s1, W2, b2)
cost = MSE(y, l2)

feed_dict = {
    X: X_,
    y: Y_,
    W1: W1_,
    b1: b1_,
    W2: W2_,
    b2: b2_
}

In [26]:
# train
epochs = 20
# examples
m = X_.shape[0]
batch_size = 11
steps_per_epoch = m // batch_size

print("Total number of examples = {}".format(m))

graph = topological_sort(feed_dict)
trainables = [W1, b1, W2, b2]

for i in range(epochs):
    loss = 0
    for j in range(steps_per_epoch):
        # randomly sample a batch
        X_batch, y_batch = resample(X_, Y_, n_samples=batch_size)
        
        # reset X and Y inputs
        X.value = X_batch
        y.value = y_batch
        
        forward_and_backward(graph)
        
        sgd_update(trainables)
        
        loss += graph[-1].value
    
    print("Epoch: {}, Loss: {:.3f}".format(i+1, loss/steps_per_epoch))

Total number of examples = 506
Epoch: 1, Loss: 132.476
Epoch: 2, Loss: 35.604
Epoch: 3, Loss: 28.237
Epoch: 4, Loss: 22.203
Epoch: 5, Loss: 24.038
Epoch: 6, Loss: 19.538
Epoch: 7, Loss: 24.474
Epoch: 8, Loss: 20.075
Epoch: 9, Loss: 18.435
Epoch: 10, Loss: 14.405
Epoch: 11, Loss: 14.093
Epoch: 12, Loss: 18.494
Epoch: 13, Loss: 16.675
Epoch: 14, Loss: 14.481
Epoch: 15, Loss: 14.326
Epoch: 16, Loss: 10.847
Epoch: 17, Loss: 11.578
Epoch: 18, Loss: 12.656
Epoch: 19, Loss: 13.259
Epoch: 20, Loss: 12.627
